In [1]:
import collections
import math
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import nltk

from nltk import FreqDist
from nltk.util import ngrams
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords
import numpy as np
import string

import matplotlib.pyplot as plt

In [2]:
try:
    import cPickle as pickle
except ImportError:  # python 3.x
    import pickle
    
# importing the index from the dictionary.p file

with open('dictionary.p', 'rb') as fp:
    d = pickle.load(fp)

In [4]:
#This code is just used for finally returning the documents. These data structures (list of documents) are not used in evaluating the queries

ag40 = open('wiki_40', encoding="utf8")

soup = BeautifulSoup(ag40)

docList = soup.findAll('doc')
numberDocs = len(docList)
docs = []

for i in range(numberDocs):
    soupTemp = BeautifulSoup(str(docList[i]))
    txtTemp = soupTemp.get_text()
    docs.append(txtTemp)
    
import re

for i in range(numberDocs):
    docs[i] = docs[i].lower()
    docs[i] = re.sub(r'[^\w\s]','',docs[i])

In [5]:
# function to check whether any of the query terms is in the mentioned document

def queryInDoc(tokens,d,docID):
    for key in tokens:
        if key in d.keys():
            travlist = d[key]
            #print(travlist)
            for i in range(len(travlist)):
                if(travlist[i][0] == docID):
                    return True
    return False

In [6]:
def queryInDoc2(tokens,d,docID): #returns true for those docs which contain atleast 70% of the query terms
    countDoc = 0
    countTokens = len(tokens)
    
    for i in range(countTokens):
        #traverse the posting list of token[i]
        if tokens[i] in d.keys():
            tempList = d[tokens[i]]
            for k in tempList:
                if k[0] == docID:
                    countDoc = countDoc + 1
                    break
        #else: #this query token is not in any of the documents   
        
    if countDoc/countTokens >= 0.7:
        return True
    else:
        return False

In [7]:
#frequency of a word in a document

def wordFreq(key,d,docID):
    if key in d.keys():
        travlist = d[key]
        for i in range(len(travlist)):
            if(travlist[i][0] == docID):
                return travlist[i][1]
    return 0

In [8]:
# function to return a unique token dictionary for a document

def uniqueTokens(document):
    tokens = nltk.tokenize.word_tokenize(document)
    tempDict1 = {}
    uniqueTokens = []
    for i in range(len(tokens)):
        if tokens[i] in tempDict1.keys():
            tempDict1[tokens[i]] = tempDict1[tokens[i]] + 1
        else:
            tempDict1[tokens[i]] = 1
    return tempDict1

In [78]:
queries = []
queries.append('the modern re-interpretations of classical music')
queries.append('As in the Five Hoosier Painters')
queries.append('Highway construction')
queries.append('school of drama')
queries.append('idhar query 5 daalna')
queries.append('idhar query 6 daalna')
queries.append('Chief Executive Officer')
queries.append('medieval architecture')
queries.append('university teacher')
queries.append('seven quality coach')

In [14]:
def returnTopDocs(d,query):
    query = query.lower()
    query = re.sub(r'[^\w\s]','',query)
    query_tokens = nltk.tokenize.word_tokenize(query)
    scores = []
    queryDict = {}
    
    for j in range(len(query_tokens)):
        if query_tokens[j] in queryDict.keys():
            queryDict[query_tokens[j]] = queryDict[query_tokens[j]] + 1
        else:
            queryDict[query_tokens[j]] = 1
    
    keys = list(queryDict.keys())
    tf = []
    idf = []
    wt = []
    for i in range(len(keys)):
        tf.append(queryDict[keys[i]])
        if(keys[i] in d.keys()):
            idf.append(1 + math.log2(numberDocs/len(d[keys[i]])))
        else:
            idf.append(0)
        wt.append(tf[i]*idf[i])
    
    sumQuerySquare = 0
    for i in range(len(wt)):
        sumQuerySquare = sumQuerySquare + wt[i]*wt[i]
    sumQueryRoot = math.sqrt(sumQuerySquare)
    
    for i in range(len(wt)):
        wt[i] = wt[i]/sumQueryRoot
    
    queryKeysWts = {}
    for i in range(len(keys)):
        queryKeysWts[keys[i]] = wt[i]
        
    docScores = []
    
    for i in range(numberDocs):
        if queryInDoc(query_tokens,d,i):
            docDict = uniqueTokens(docs[i])
            docDict2 = {}
            docSum = 0
            for k in docDict:
                tfd = docDict[k]
                tfdwt = 1 + math.log2(tfd)
                docSum = docSum + tfdwt*tfdwt
                docDict2[k] = tfdwt
            
            docSum = math.sqrt(docSum)
            
            docScore = 0
            for m in range(len(keys)):
                token1 = keys[m]
                if token1 in docDict:
                    docScore = docScore + (docDict2[token1]/docSum)*wt[m]
            
            docScores.append([docScore,i])
    
    #docScores = docScores.sort(reverse=True)
    return docScores

In [79]:
r = returnTopDocs(d,queries[9])
r = sorted(r,reverse=True)
len(r)

329

In [80]:
r

[[0.207805907294367, 68],
 [0.13113833524331556, 1076],
 [0.1234175640890745, 1754],
 [0.11578844409676164, 1442],
 [0.11161599663132674, 1619],
 [0.10809777561352757, 28],
 [0.10741571599651598, 3180],
 [0.10659096869161729, 3445],
 [0.1060372470842922, 180],
 [0.09998977908425799, 1314],
 [0.09830713012883364, 3272],
 [0.09472720442856188, 957],
 [0.09282185557170992, 1726],
 [0.09264508049639647, 329],
 [0.09216849510028358, 2080],
 [0.09041490559782114, 1201],
 [0.08848847983226679, 95],
 [0.08721561914775114, 2495],
 [0.08705911540694618, 2879],
 [0.08293764576961803, 2772],
 [0.08293764576961803, 2643],
 [0.08096657758443528, 2040],
 [0.0805590637432226, 2741],
 [0.080341848720751, 2607],
 [0.08030623982706937, 988],
 [0.0789295205440991, 1482],
 [0.07891443755441761, 1724],
 [0.07865835683242706, 1732],
 [0.07755712807047181, 377],
 [0.07751374357848474, 1743],
 [0.07472468207827054, 844],
 [0.07453380456695043, 102],
 [0.07451659253979342, 1912],
 [0.07413045637686629, 2987],
 

In [82]:
docs[1076]

'\nedgar wingard\n\nedgar ramey wingard september 21 1878  july 31 1927 was an american football basketball and baseball coach and college athletics administrator he served as the head football coach at seven different schools ohio northern university 1903 butler university 19041905 the university of pittsburgh 1906 louisiana state university 19071908 the university of maine 19101911 susquehanna university 19161917 1919 19241925 and bucknell university 1918 compling a career record of 77395 in 1908 wingard led his lsu team to a record of 100 the team has been recognized as a national champion by the national championship foundation although lsu does not officially claim a national title that season wingard was the head coach of the basketball team at butler from 1904 to 1906 and the head coach of the first lsu tigers basketball team during the 190809 season he also coached the lsu tigers baseball team in 1908 and 1909 and the baseball team at maine in 1911\n\nwingard died of a cerebral

In [16]:
def idfofterm(word, dictionary, N): #N is the number of Documents total 
    if word in dictionary.keys():
        return math.log2(N / len(dictionary[word]))
    else:
        return -1 #returns -1 if the idf is +infinity (word not in any document)


def returnTopDocsHIGHIDF(d,query):
    query = query.lower()
    query = re.sub(r'[^\w\s]','',query)
    query_tokens = nltk.tokenize.word_tokenize(query)
    scores = []
    queryDict = {}
    
    #print('The length of [query_tokens] is')
    #print(len(query_tokens))
    #print(query_tokens)
    
    toDelete = {}
    #removing the lowidf terms from query_tokens (all instances)
    for term in query_tokens:
        if idfofterm(term, d, numberDocs)<2.00:
            if term not in toDelete.keys():
                toDelete[term] = 1
    
    for key in toDelete:
        while key in query_tokens: query_tokens.remove(key)

    print('The length of [query_tokens] without lowIDF terms is')
    print(len(query_tokens)) 
    print(query_tokens)
    
    
    for j in range(len(query_tokens)):
        if query_tokens[j] in queryDict.keys():
            queryDict[query_tokens[j]] = queryDict[query_tokens[j]] + 1
        else:
            queryDict[query_tokens[j]] = 1
      
    keys = list(queryDict.keys())
    tf = []
    idf = []
    wt = []
    for i in range(len(keys)):
        tf.append(queryDict[keys[i]])
        if(keys[i] in d.keys()):
            idf.append(1 + math.log2(numberDocs/len(d[keys[i]])))
        else:
            idf.append(0)
        wt.append(tf[i]*idf[i])
    
    sumQuerySquare = 0
    for i in range(len(wt)):
        sumQuerySquare = sumQuerySquare + wt[i]*wt[i]
    sumQueryRoot = math.sqrt(sumQuerySquare)
    
    for i in range(len(wt)):
        wt[i] = wt[i]/sumQueryRoot
    
    queryKeysWts = {}
    for i in range(len(keys)):
        queryKeysWts[keys[i]] = wt[i]
        
    docScores = []
    
    for i in range(numberDocs):
        if queryInDoc(query_tokens,d,i):
            docDict = uniqueTokens(docs[i])
            docDict2 = {}
            docSum = 0
            for k in docDict:
                tfd = docDict[k]
                tfdwt = 1 + math.log2(tfd)
                docSum = docSum + tfdwt*tfdwt
                docDict2[k] = tfdwt
            
            docSum = math.sqrt(docSum)
            
            docScore = 0
            for m in range(len(keys)):
                token1 = keys[m]
                if token1 in docDict:
                    docScore = docScore + (docDict2[token1]/docSum)*wt[m]
            
            docScores.append([docScore,i])
    
    #docScores = docScores.sort(reverse=True)
    return docScores

In [18]:
def returnTopDocsMANYQUERYTERMS(d,query):
    query = query.lower()
    query = re.sub(r'[^\w\s]','',query)
    query_tokens = nltk.tokenize.word_tokenize(query)
    scores = []
    queryDict = {}

    for j in range(len(query_tokens)):
        if query_tokens[j] in queryDict.keys():
            queryDict[query_tokens[j]] = queryDict[query_tokens[j]] + 1
        else:
            queryDict[query_tokens[j]] = 1
    
    keys = list(queryDict.keys())
    tf = []
    idf = []
    wt = []
    for i in range(len(keys)):
        tf.append(queryDict[keys[i]])
        if(keys[i] in d.keys()):
            idf.append(1 + math.log2(numberDocs/len(d[keys[i]])))
        else:
            idf.append(0)
        wt.append(tf[i]*idf[i])
    
    sumQuerySquare = 0
    for i in range(len(wt)):
        sumQuerySquare = sumQuerySquare + wt[i]*wt[i]
    sumQueryRoot = math.sqrt(sumQuerySquare)
    
    for i in range(len(wt)):
        wt[i] = wt[i]/sumQueryRoot
    
    queryKeysWts = {}
    for i in range(len(keys)):
        queryKeysWts[keys[i]] = wt[i]
        
    docScores = []
    
    for i in range(numberDocs):
        if queryInDoc2(query_tokens,d,i):
            docDict = uniqueTokens(docs[i])
            docDict2 = {}
            docSum = 0
            for k in docDict:
                tfd = docDict[k]
                tfdwt = 1 + math.log2(tfd)
                docSum = docSum + tfdwt*tfdwt
                docDict2[k] = tfdwt
            
            docSum = math.sqrt(docSum)
            
            docScore = 0
            for m in range(len(keys)):
                token1 = keys[m]
                if token1 in docDict:
                    docScore = docScore + (docDict2[token1]/docSum)*wt[m]
            
            docScores.append([docScore,i])
    
    #docScores = docScores.sort(reverse=True)
    return docScores

In [19]:
def returnTopDocsBOTH(d,query):
    query = query.lower()
    query = re.sub(r'[^\w\s]','',query)
    query_tokens = nltk.tokenize.word_tokenize(query)
    scores = []
    queryDict = {}
    
    print('The length of [query_tokens] is')
    print(len(query_tokens))
    print(query_tokens)
    
    toDelete = {}
    #removing the lowidf terms from query_tokens (all instances)
    for term in query_tokens:
        if idfofterm(term, d, numberDocs)<2.00:
            if term not in toDelete.keys():
                toDelete[term] = 1
    
    for key in toDelete:
        while key in query_tokens: query_tokens.remove(key)

    print('The length of [query_tokens] without lowIDF terms is')
    print(len(query_tokens)) 
    print(query_tokens)
    
    
    for j in range(len(query_tokens)):
        if query_tokens[j] in queryDict.keys():
            queryDict[query_tokens[j]] = queryDict[query_tokens[j]] + 1
        else:
            queryDict[query_tokens[j]] = 1
 
    keys = list(queryDict.keys())
    tf = []
    idf = []
    wt = []
    for i in range(len(keys)):
        tf.append(queryDict[keys[i]])
        if(keys[i] in d.keys()):
            idf.append(1 + math.log2(numberDocs/len(d[keys[i]])))
        else:
            idf.append(0)
        wt.append(tf[i]*idf[i])
    
    sumQuerySquare = 0
    for i in range(len(wt)):
        sumQuerySquare = sumQuerySquare + wt[i]*wt[i]
    sumQueryRoot = math.sqrt(sumQuerySquare)
    
    for i in range(len(wt)):
        wt[i] = wt[i]/sumQueryRoot
    
    queryKeysWts = {}
    for i in range(len(keys)):
        queryKeysWts[keys[i]] = wt[i]
        
    docScores = []
    
    for i in range(numberDocs):
        if queryInDoc2(query_tokens,d,i):
            docDict = uniqueTokens(docs[i])
            docDict2 = {}
            docSum = 0
            for k in docDict:
                tfd = docDict[k]
                tfdwt = 1 + math.log2(tfd)
                docSum = docSum + tfdwt*tfdwt
                docDict2[k] = tfdwt
            
            docSum = math.sqrt(docSum)
            
            docScore = 0
            for m in range(len(keys)):
                token1 = keys[m]
                if token1 in docDict:
                    docScore = docScore + (docDict2[token1]/docSum)*wt[m]
            
            docScores.append([docScore,i])
    
    #docScores = docScores.sort(reverse=True)
    return docScores